### Import thư viện cần thiết

In [1]:
import requests
import datetime
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from lxml import etree

### Tạo Webdriver

In [2]:
apart_arr = []
# Declare browser
options = Options()
options.add_argument("--headless")
options.add_argument("---window-size=1920,1080")
PATH = "D:\Tai_Phan_Mem\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(PATH) #, options = options)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11784\1692986675.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH) #, options = options)


### I. Khởi tạo 

#### 1. Dataframe rỗng sẽ chứa dữ liệu thu thập

In [3]:
# DATAFRAME FEATURES
# address, price, area, property_status, price_per_m2, bedroom, toilets, floornumber, property_legal, furniture
df_columns = ['address', 'area', 'project', 'price', 'property_status', 'price_per_m2', 'bedroom', 'toilet', \
              'floornumber','property_legal', 'furniture', 'balconydirection', 'direction', 'apartment_feature']
len(df_columns)
# Khởi tạo dataframe rỗng, tạo sẵn các cột
df = pd.DataFrame(columns = df_columns)
df.drop(df.index,inplace=True) 
df.shape

(0, 14)

#### 2. Lấy link (đường dẫn) đến từng chung cư cụ thể

In [4]:
# hàm lấy link của các trang chủ (num_page = số trang)
# Thêm vào
def get_apart_links(url, num_page):
    apartments_link = []
    for i in range(num_page):
        driver2 = webdriver.Chrome(PATH) #, options = options)
        driver2.get(url + str(i))
        
        # Open URL
        soup = BeautifulSoup(driver2.page_source)
        a = soup.find_all("a",class_='AdItem_adItem__gDDQT')
        for i in range(len(a)):
            link ="https://nha.chotot.com/"+a[i]['href']
            apartments_link.append(link)
    return apartments_link


Chạy trên 110 trang (mỗi trang chứa nhiều chung cư). Sau bước này sẽ được đường link dẫn đến từng chung cư cụ thể, giá trị được lưu vào biến **apartment_links**. </br>
Lưu **apartment_links** thành file **apart_link.csv** để dự phòng.


In [ ]:
url = "https://www.nhatot.com/mua-ban-can-ho-chung-cu-tp-ho-chi-minh?page="
num_page = 120
apartment_links = get_apart_links(url, num_page)

In [ ]:
apart_link_df = pd.DataFrame(data = apartment_links)
apart_link_df.to_csv('apart_link.csv', index = False, encoding='utf-8-sig')
apart_link_df.shape

In [7]:
link_df = pd.read_csv('apart_link.csv')
apartment_links = link_df['0']
print(apartment_links.shape)

(1874,)


### 3. Hàm truy cập từng chung cư/page để lấy dữ liệu

In [8]:
# Trả về thông tin (list) của 1 apartment
def get_info_of_apartment(apartment_link):
    driver1 = webdriver.Chrome(PATH) #, options = options)
    driver1.get(apartment_link)
    time.sleep(2)
    driver1.execute_script('window.scrollTo(0,845)')

    dacdiem = driver1.find_element(By.CLASS_NAME, 'AdParam_adParamTitle__bU__w')
    y_coord = dacdiem.location['y']
    driver1.execute_script("window.scrollTo(0, arguments[0])", y_coord - 60)

    try:
        xemthem = driver1.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div[1]/div/div[4]/div/div[3]/button')
        xemthem.click()
    except: 
        pass
    
    soup1 = BeautifulSoup(driver1.page_source)
    
    price = soup1.find('span', itemprop = 'price')
    price = price.text.split(' - ')[0]
    area = soup1.find('span', itemprop = 'size')
    address = soup1.find(class_ = 'fz13')
    property_status = soup1.find('span', itemprop = 'property_status')
    price_per_m2 = soup1.find('span', itemprop="price_m2")
    bedroom = soup1.find('span', itemprop="rooms")
    toilets = soup1.find('span', itemprop="toilets")
    floornumber = soup1.find('span', itemprop="floornumber")
    property_legal = soup1.find('span', itemprop="property_legal_document")
    furniture = soup1.find('span', itemprop = 'furnishing_sell')
    dom = etree.HTML(str(soup1))
    # (new code)
    project = dom.xpath('/html/body/div[1]/div/div[3]/div[1]/div/div[4]/div/div[2]/div/div/div/span/div/div[3]/div/div/div[2]/span/span[2]/a/text()')
    balconydirection = soup1.find('span', itemprop = 'balconydirection')
    direction = soup1.find('span', itemprop = 'direction')
    apartment_feature = soup1.find('span', itemprop = 'apartment_feature')
    
    info_list = [] # HTML type
    info_text = [] # text
    info_list.extend([address, price, project, area, property_status, price_per_m2, bedroom, toilets, floornumber, \
                      property_legal, furniture, balconydirection, direction, apartment_feature])
    info_text.extend([address.text[:-10], price])
    if (len(project) != 0):
        info_text.extend([project[0]])
        
    
    for feature in info_list[3:]:
        if feature != None:
            info_text.append(feature.text)
        else:
            info_text.append('')
    driver1.quit()
    return info_text
    


In [9]:
# Lấy thông tin từng trang, thêm vào df
def get_info_of_page(apart_arr):
    df_temp = df.copy()
    for apart in apart_arr:
        try:
            apart_info = get_info_of_apartment(apart)
            df_temp.loc[len(df_temp)] = apart_info
        except:
            continue
    return df_temp


### II. Thu thập dữ liệu của các chung cư
Truy cập từng link để lấy thông tin của từng chung cư cụ thể. Chạy trên từng 100 links chung cư, sau đó lưu thành dataframe (df1, df2,..., df17). Lưu lại file csv để dự phòng.

In [10]:
df1 = get_info_of_page(apartment_links[0:100])
df1.to_csv('new-data-1.csv', index = False, encoding='utf-8-sig')
df1.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

(79, 14)

In [11]:
df2 = get_info_of_page(apartment_links[100:200])
df2.to_csv('new-data-2.csv', index = False)

In [12]:
df3 = get_info_of_page(apartment_links[200:300])
df3.to_csv('new-data-3.csv', index = False, encoding='utf-8-sig')
df3.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

(85, 14)

In [13]:
df4 = get_info_of_page(apartment_links[300:400])
df4.to_csv('new-data-4.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [15]:
df5 = get_info_of_page(apartment_links[400:500])
df5.to_csv('new-data-5.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [16]:
df6 = get_info_of_page(apartment_links[500:600])
df6.to_csv('new-data-6.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [17]:
df7 = get_info_of_page(apartment_links[600:700])
df7.to_csv('new-data-7.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [18]:
df8 = get_info_of_page(apartment_links[700:800])
df8.to_csv('new-data-8.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [19]:
df9 = get_info_of_page(apartment_links[800:900])
df9.to_csv('new-data-9.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [20]:
df10 = get_info_of_page(apartment_links[900:1000])
df10.to_csv('new-data-10.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [21]:
df11 = get_info_of_page(apartment_links[1000:1100])
df11.to_csv('new-data-11.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [22]:
df12 = get_info_of_page(apartment_links[1100:1200])
df12.to_csv('new-data-12.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [23]:
df13 = get_info_of_page(apartment_links[1200:1274])
df13.to_csv('new-data-13.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [14]:
df14 = get_info_of_page(apartment_links[1274: 1474])
df14.to_csv('new-data-14.csv', index = False, encoding='utf-8-sig') 

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\2657334244.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [62]:
df15 = get_info_of_page(apartment_links[1474:1674])
df15.to_csv('new-data-15.csv', index = False, encoding='utf-8-sig')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11360\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

In [58]:
df16 = get_info_of_page(apartment_links[1674:1874])
df16.to_csv('new-data-16.csv', index = False, encoding='utf-8-sig')
df16.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_62076\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_62076\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_62076\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_62076\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_62076\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

(168, 14)

In [13]:
df17 = get_info_of_page(apartment_links[-200:])
df17.to_csv('new-data-17.csv', index = False, encoding='utf-8-sig')
df17.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_11784\1231544838.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11784\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11784\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11784\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11784\1949831379.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service

(158, 14)

### III. Lưu kết quả
Đọc lại các file csv đã lưu sau khi lấy dữ liệu để tổng hợp thành file **chotot-final.csv**.

In [39]:
df1 = pd.read_csv('new-data-1.csv')
df2 = pd.read_csv('new-data-2.csv')
df3 = pd.read_csv('new-data-3.csv')
df4 = pd.read_csv('new-data-4.csv')
df5 = pd.read_csv('new-data-5.csv')
df6 = pd.read_csv('new-data-6.csv')
df7 = pd.read_csv('new-data-7.csv')
df8 = pd.read_csv('new-data-8.csv')
df9 = pd.read_csv('new-data-9.csv')
df10 = pd.read_csv('new-data-10.csv')
df11 = pd.read_csv('new-data-11.csv')
df12 = pd.read_csv('new-data-12.csv')
df13 = pd.read_csv('new-data-13.csv')
df14 = pd.read_csv('new-data-14.csv')
df15 = pd.read_csv('new-data-15.csv')
df16 = pd.read_csv('new-data-16.csv')
df17 = pd.read_csv('new-data-17.csv')
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)
print(df6.shape)
print(df7.shape)
print(df8.shape)
print(df9.shape)
print(df10.shape)
print(df11.shape)
print(df12.shape)
print(df13.shape)
print(df14.shape)
print(df15.shape)
print(df16.shape)
print(df17.shape)

(79, 14)
(74, 14)
(85, 14)
(74, 14)
(67, 14)
(84, 14)
(74, 14)
(80, 14)
(80, 14)
(86, 14)
(76, 14)
(77, 14)
(51, 14)
(152, 14)
(163, 14)
(168, 14)
(158, 14)


In [42]:
df_data = pd.concat([df1, df2, df3, df4, df5, df6, df7,df8,df9,df10,df11,df12, df13, df14, df15, df16, df17], axis = 0)
df_data = df_data.drop_duplicates()
print(df_data.shape)
df_data.to_csv('chotot-final.csv', index = False, encoding='utf-8-sig')

(1137, 14)


In [43]:
df_data.head()

,address,area,project,price,property_status,price_per_m2,bedroom,toilet,floornumber,property_legal,furniture,balconydirection,direction,apartment_feature
0,"Đường Huy Cận, Phường Phước Long B (Quận 9 cũ)...","2,35 tỷ",The Art,66 m²,Chưa bàn giao,"35,61 triệu/m²",2 phòng,2 phòng,8.0,Đã có sổ,Nội thất cao cấp,Tây Bắc,Đông Bắc,NaN
1,"số 88, số 88 Đường N1, Phường Sơn Kỳ, Quận Tân...","3,42 tỷ",Celadon City,71 m²,Đã bàn giao,"48,17 triệu/m²",2 phòng,2 phòng,NaN,Đang chờ sổ,Nội thất cao cấp,NaN,NaN,NaN
2,"Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân, Tp ...","2,4 tỷ",Akari City,56 m²,Đã bàn giao,"42,86 triệu/m²",2 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Bát Nàn, Phường Thạnh Mỹ Lợi (Quận 2 cũ), Thàn...","7,5 tỷ",One Verandah Mapletree,107 m²,Đã bàn giao,"70,09 triệu/m²",3 phòng,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Võ Văn Kiệt, Phường An Lạc, Quận Bình Tân, Tp ...","3,1 tỷ",Akari City,75 m²,Chưa bàn giao,"41,33 triệu/m²",2 phòng,2 phòng,NaN,NaN,NaN,NaN,NaN,NaN
